In [1]:
from src.schemas.thesaurus.mads import SchemaMads
from datetime import datetime
from src.schemas.settings import Settings
from rdflib import URIRef, Namespace, Graph, Literal, BNode
from rdflib.namespace import RDF, RDFS
settings = Settings()

In [2]:
req = {
    "adminMetadata": {
        "status": {
            "value": "http://id.loc.gov/vocabulary/mstatus/n",
            "label": "Novo"
        },
        "descriptionConventions": {
            "value": "http://id.loc.gov/vocabulary/descriptionConventions/aacr",
            "label": "AACr"
        }
    },
    "resource": [
        {
            "type": {
                "value": "http://www.loc.gov/mads/rdf/v1#Authority",
                "label": "Authority"
            }
        },
        {
            "type": {
                "value": "http://www.loc.gov/mads/rdf/v1#Topic",
                "label": "Topic"
            }
        }
    ],
    "isMemberOfMADSCollection": [
        {
            "collection": {
                "value": "subjects",
                "label": "Assuntos"
            }
        }
    ],
    "elementList": [
        {
            "elementType": {
                "value": "http://www.loc.gov/mads/rdf/v1#TopicElement",
                "label": "TopicElement"
            },
            "elementValue": "Rain forests",
            "elementLang": {
                "value": "en",
                "label": "Inglês"
            }
        }
    ],
    "authoritativeLabel": {
        "value": "Rain forests"
    },
    "hasBroaderAuthority": [
        {
            "base": "loc",
            "uri": "http://id.loc.gov/authorities/subjects/sh85050707",
            "label": "Forests and forestry",
            "elementLang": {
                "value": "en",
                "label": "Inglês"
            }
        }
    ],
    "hasVariant": [
        {
            "typeVariant": {
                "value": "http://www.loc.gov/mads/rdf/v1#ComplexSubject",
                "label": "ComplexSubject"
            },
            "elementList": [
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#Topic",
                        "label": "Topic"
                    },
                    "elementValue": "Rain forests",
                    "elementLang": {
                        "value": "en",
                        "label": "Inglês"
                    }
                },
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#Geographic",
                        "label": "Geographic"
                    },
                    "elementValue": "Tropics",
                    "elementLang": {
                        "value": "en",
                        "label": "Inglês"
                    }
                }
            ]
        },
        {
            "typeVariant": {
                "value": "http://www.loc.gov/mads/rdf/v1#Topic",
                "label": "Topic"
            },
            "elementList": [
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#TopicElement",
                        "label": "TopicElement"
                    },
                    "elementValue": "Rainforests",
                    "elementLang": {
                        "value": "en",
                        "label": "Inglês"
                    }
                }
            ]
        },
        {
            "typeVariant": {
                "value": "http://www.loc.gov/mads/rdf/v1#Topic",
                "label": "Topic"
            },
            "elementList": [
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#TopicElement",
                        "label": "TopicElement"
                    },
                    "elementValue": "Tropical rain forests",
                    "elementLang": {
                        "value": "en",
                        "label": "Inglês"
                    }
                }
            ]
        },
        {
            "typeVariant": {
                "value": "http://www.loc.gov/mads/rdf/v1#Topic",
                "label": "Topic"
            },
            "elementList": [
                {
                    "elementType": {
                        "value": "http://www.loc.gov/mads/rdf/v1#TopicElement",
                        "label": "TopicElement"
                    },
                    "elementValue": "Tropical rainforests",
                    "elementLang": {
                        "value": "en",
                        "label": "Inglês"
                    }
                }
            ]
        }
    ],
    "hasCloseExternalAuthority": [
        {
            "base": "d-nb.info",
            "label": "Tropischer Regenwald",
            "uri": "https://d-nb.info/gnd/4061016-0"
        },
        {
            "base": "data.bnf.fr",
            "label": "Forêts pluviales",
            "uri": "http://data.bnf.fr/ark:/12148/cb11931438h"
        },
        {
            "base": "id.ndl.go.jp",
            "label": "熱帯林",
            "uri": "http://id.ndl.go.jp/auth/ndlsh/00568116"
        },
        {
            "base": "purl.org",
            "label": "Foreste equatoriali",
            "uri": "http://purl.org/bncf/tid/6301"
        },
        {
            "base": "www.yso.fi",
            "label": "sademetsät",
            "uri": "http://www.yso.fi/onto/yso/p3487"
        },
        {
            "base": "d-nb.info",
            "label": "Regenwald",
            "uri": "https://d-nb.info/gnd/4131965-5"
        },
        {
            "base": "id.worldcat.org",
            "label": "Rain forests",
            "uri": "http://id.worldcat.org/fast/1089466"
        },
        {
            "base": "vocab.getty.edu",
            "label": "rainforests",
            "uri": "http://vocab.getty.edu/aat/300008871"
        },
        {
            "base": "www.wikidata.org",
            "label": "rainforest",
            "uri": "http://www.wikidata.org/entity/Q9444"
        },
        {
            "base": "datos.bne.es",
            "label": "Bosques tropicales",
            "uri": "http://datos.bne.es/resource/XX535690"
        }
    ],
    "identifiersLccn": "sh85111193"
}

In [3]:
request = SchemaMads(**req)
now = datetime.now()
request.adminMetadata.creationDate = now
request.adminMetadata.identifiedBy = "teste"

In [4]:
request.hasBroaderAuthority

[Uri(base='loc', uri='http://id.loc.gov/authorities/subjects/sh85050707', label='Forests and forestry', elementLang=Element(value='en', label='Inglês'))]

In [7]:
MDASRDF = Namespace("http://www.loc.gov/mads/rdf/v1#")
BF = Namespace("http://id.loc.gov/ontologies/bibframe/")
IDENTIFIERS = Namespace("http://id.loc.gov/vocabulary/identifiers/")
identifier = f"{settings.base_url}/authorities/{request.adminMetadata.identifiedBy}"
resource = URIRef(identifier)

g = Graph(identifier=identifier)
g.bind('madsrdf', MDASRDF)
g.bind('identifiers', IDENTIFIERS)
g.bind('bf', BF)


In [10]:
def HasBroader(g, resource, hasBroader, MDASRDF):
    for i in hasBroader:
        authority = URIRef(i.uri)
        g.add((authority, RDF.type, MDASRDF.Authority))
        g.add((authority, RDF.type, MDASRDF.Topic))
        g.add((authority, MDASRDF.authoritativeLabel, Literal(i.label, lang=i.elementLang.value)))
        g.add((resource, MDASRDF.hasBroaderAuthority, authority))
    return g

if request.hasBroaderAuthority:
    g = HasBroader(g, resource, request.hasBroaderAuthority, MDASRDF)
    # 

In [12]:
print(g.serialize())

@prefix madsrdf: <http://www.loc.gov/mads/rdf/v1#> .

<https://bibliokeia.com/authorities/teste> madsrdf:hasBroaderAuthority <http://id.loc.gov/authorities/subjects/sh85050707> ;
    madsrdf:hasCloseExternalAuthority <http://id.loc.gov/authorities/subjects/sh85050707> .

<http://id.loc.gov/authorities/subjects/sh85050707> a madsrdf:Authority,
        madsrdf:Topic ;
    madsrdf:authoritativeLabel "Forests and forestry",
        "Forests and forestry"@en .




In [13]:
from pysolr import Solr
solr = Solr(f'{settings.solr}/solr/authority/', timeout=10)

In [14]:
responseSolr = solr.add([doc], commit=True)
responseSolr

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":146\n  }\n}'

In [4]:
from src.function.thesaurus.solr.updateSolr import UpdateSolr


UpdateSolr(request)